In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Ensure eager execution
tf.config.run_functions_eagerly(True)

# Dataset (full list as before, truncated here)
english_to_french = [
    ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur"),
    ("They ride bicycles", "Ils font du vélo"),
    ("The coffee is hot", "Le café est chaud"),
    ("She wears glasses", "Elle porte des lunettes"),
    ("We visit our grandparents", "Nous rendons visite à nos grands-parents"),
    ("He plays the guitar", "Il joue de la guitare"),
    ("They go shopping", "Ils font du shopping"),
    ("The teacher explains the lesson", "Le professeur explique la leçon"),
    ("She takes the train to work", "Elle prend le train pour aller au travail"),
    ("We bake cookies", "Nous faisons des biscuits"),
    ("He washes his hands", "Il se lave les mains"),
    ("They enjoy the sunset", "Ils apprécient le coucher du soleil"),
    ("The river flows calmly", "La rivière coule calmement"),
    ("She feeds the cat", "Elle nourrit le chat"),
    ("We visit the museum", "Nous visitons le musée"),
    ("He fixes his bicycle", "Il répare son vélo"),
    ("They paint the walls", "Ils peignent les murs"),
    ("The baby sleeps peacefully", "Le bébé dort paisiblement"),
    ("She ties her shoelaces", "Elle attache ses lacets"),
    ("We climb the stairs", "Nous montons les escaliers"),
    ("He shaves in the morning", "Il se rase le matin"),
    ("They set the table", "Ils mettent la table"),
    ("The airplane takes off", "L'avion décolle"),
    ("She waters the plants", "Elle arrose les plantes"),
    ("We practice yoga", "Nous pratiquons le yoga"),
    ("He turns off the light", "Il éteint la lumière"),
    ("They play video games", "Ils jouent aux jeux vidéo"),
    ("The soup smells delicious", "La soupe sent délicieusement bon"),
    ("She locks the door", "Elle ferme la porte à clé"),
    ("We enjoy a picnic", "Nous profitons d'un pique-nique"),
    ("He checks his email", "Il vérifie ses emails"),
    ("They go to the gym", "Ils vont à la salle de sport"),
    ("The moon shines brightly", "La lune brille intensément"),
    ("She catches the bus", "Elle attrape le bus"),
    ("We greet our neighbors", "Nous saluons nos voisins"),
    ("He combs his hair", "Il se peigne les cheveux"),
    ("They wave goodbye", "Ils font un signe d'adieu")

]

# Split into English and French
english_sentences, french_sentences = zip(*english_to_french)

# Add <SOS> and <EOS> tokens to French sentences
french_sentences = ['<SOS> ' + sent + ' <EOS>' for sent in french_sentences]

# Tokenization
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_sentences)
eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)
eng_vocab_size = len(eng_tokenizer.word_index) + 1

fr_tokenizer = Tokenizer(filters='')
fr_tokenizer.fit_on_texts(french_sentences)

# Manually ensure <SOS> and <EOS> are in the vocabulary
if '<SOS>' not in fr_tokenizer.word_index:
    fr_tokenizer.word_index['<SOS>'] = len(fr_tokenizer.word_index) + 1
if '<EOS>' not in fr_tokenizer.word_index:
    fr_tokenizer.word_index['<EOS>'] = len(fr_tokenizer.word_index) + 1

fr_sequences = fr_tokenizer.texts_to_sequences(french_sentences)
fr_vocab_size = len(fr_tokenizer.word_index) + 1

# Debug: Inspect the vocabulary
print("French tokenizer word_index:", fr_tokenizer.word_index)

# Padding sequences
max_eng_len = max(len(seq) for seq in eng_sequences)
max_fr_len = max(len(seq) for seq in fr_sequences)
eng_padded = pad_sequences(eng_sequences, maxlen=max_eng_len, padding='post')
fr_padded = pad_sequences(fr_sequences, maxlen=max_fr_len, padding='post')

# Prepare decoder input and target data
decoder_input_data = fr_padded[:, :-1]  # Exclude <EOS>
decoder_target_data = fr_padded[:, 1:]  # Exclude <SOS>

# Model Parameters
embedding_dim = 256
units = 512

# Encoder
encoder_inputs = Input(shape=(max_eng_len,))
enc_embedding = Embedding(eng_vocab_size, embedding_dim)(encoder_inputs)
encoder_gru = GRU(units, return_state=True)
encoder_outputs, encoder_state = encoder_gru(enc_embedding)

# Decoder
decoder_inputs = Input(shape=(max_fr_len - 1,))
dec_embedding = Embedding(fr_vocab_size, embedding_dim)(decoder_inputs)
decoder_gru = GRU(units, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(dec_embedding, initial_state=encoder_state)
decoder_dense = Dense(fr_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train and evaluate on the entire dataset
history = model.fit(
    [eng_padded, decoder_input_data],
    np.expand_dims(decoder_target_data, -1),
    batch_size=32,
    epochs=50,
    validation_data=([eng_padded, decoder_input_data], np.expand_dims(decoder_target_data, -1)),
    verbose=1
)

# Report results
print("Training Loss:", history.history['loss'][-1])
print("Validation Loss (Entire Dataset):", history.history['val_loss'][-1])
print("Validation Accuracy (Entire Dataset):", history.history['val_accuracy'][-1])

# Inference Setup
encoder_model = Model(encoder_inputs, encoder_state)

decoder_state_input = Input(shape=(units,))
decoder_inputs_single = Input(shape=(1,))
dec_embedding_single = Embedding(fr_vocab_size, embedding_dim)(decoder_inputs_single)
decoder_outputs, decoder_state = decoder_gru(dec_embedding_single, initial_state=decoder_state_input)
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs_single, decoder_state_input], [decoder_outputs, decoder_state])

# Translation Function (improved to stop at first <EOS>)
def translate(sentence):
    seq = eng_tokenizer.texts_to_sequences([sentence])
    seq = pad_sequences(seq, maxlen=max_eng_len, padding='post')
    state = encoder_model.predict(seq, verbose=0)

    # Use <SOS> as defined in the vocabulary
    target_seq = np.array([[fr_tokenizer.word_index['<SOS>']]])
    output_sentence = []

    for _ in range(max_fr_len):
        output_tokens, state = decoder_model.predict([target_seq, state], verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, 0, :])
        sampled_word = fr_tokenizer.index_word.get(sampled_token_index, '')

        if sampled_word == '<EOS>':
            break
        if sampled_word:
            output_sentence.append(sampled_word)
        target_seq = np.array([[sampled_token_index]])

    return ' '.join(output_sentence)

# Qualitative Validation
test_sentences = [
    "I am cold",
    "She is happy",
    "We love music",
    "He drives a blue car",
    "The sun is shining"
]

print("\nQualitative Validation:")
for sent in test_sentences:
    translation = translate(sent)
    print(f"English: {sent} -> French: {translation}")

# Reset eager execution (optional)
tf.config.run_functions_eagerly(False)

French tokenizer word_index: {'<sos>': 1, '<eos>': 2, 'le': 3, 'la': 4, 'il': 5, 'elle': 6, 'nous': 7, 'ils': 8, 'les': 9, 'de': 10, 'à': 11, 'des': 12, 'au': 13, 'dans': 14, 'du': 15, 'une': 16, 'un': 17, 'se': 18, 'font': 19, 'est': 20, 'chat': 21, 'dort': 22, 'soleil': 23, 'musique': 24, 'jouent': 25, 'porte': 26, 'ensemble': 27, 'étudie': 28, 'bruyamment': 29, 'chante': 30, 'matin': 31, 'pour': 32, 'brille': 33, 'livres': 34, 'chaque': 35, 'film': 36, 'dîner': 37, 'voiture': 38, 'visitent': 39, 'cuisine': 40, 'regardons': 41, 'en': 42, 'voyagent': 43, 'table': 44, 'danse': 45, 'avec': 46, 'dur': 47, 'fleurs': 48, 'écrit': 49, 'son': 50, 'peint': 51, 'parc': 52, 'train': 53, 'bébé': 54, 'répare': 55, 'café': 56, 'doucement': 57, 'bus': 58, 'vélo': 59, 'nos': 60, 'ses': 61, "j'ai": 62, 'froid': 63, 'tu': 64, 'es': 65, 'fatigué': 66, 'a': 67, 'faim': 68, 'heureuse': 69, 'sommes': 70, 'amis': 71, 'sont': 72, 'étudiants': 73, 'aimons': 74, 'parle': 75, 'français': 76, 'couramment': 77, 

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Ensure eager execution
tf.config.run_functions_eagerly(True)

# Dataset
english_to_french = [
   ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur"),
    ("They ride bicycles", "Ils font du vélo"),
    ("The coffee is hot", "Le café est chaud"),
    ("She wears glasses", "Elle porte des lunettes"),
    ("We visit our grandparents", "Nous rendons visite à nos grands-parents"),
    ("He plays the guitar", "Il joue de la guitare"),
    ("They go shopping", "Ils font du shopping"),
    ("The teacher explains the lesson", "Le professeur explique la leçon"),
    ("She takes the train to work", "Elle prend le train pour aller au travail"),
    ("We bake cookies", "Nous faisons des biscuits"),
    ("He washes his hands", "Il se lave les mains"),
    ("They enjoy the sunset", "Ils apprécient le coucher du soleil"),
    ("The river flows calmly", "La rivière coule calmement"),
    ("She feeds the cat", "Elle nourrit le chat"),
    ("We visit the museum", "Nous visitons le musée"),
    ("He fixes his bicycle", "Il répare son vélo"),
    ("They paint the walls", "Ils peignent les murs"),
    ("The baby sleeps peacefully", "Le bébé dort paisiblement"),
    ("She ties her shoelaces", "Elle attache ses lacets"),
    ("We climb the stairs", "Nous montons les escaliers"),
    ("He shaves in the morning", "Il se rase le matin"),
    ("They set the table", "Ils mettent la table"),
    ("The airplane takes off", "L'avion décolle"),
    ("She waters the plants", "Elle arrose les plantes"),
    ("We practice yoga", "Nous pratiquons le yoga"),
    ("He turns off the light", "Il éteint la lumière"),
    ("They play video games", "Ils jouent aux jeux vidéo"),
    ("The soup smells delicious", "La soupe sent délicieusement bon"),
    ("She locks the door", "Elle ferme la porte à clé"),
    ("We enjoy a picnic", "Nous profitons d'un pique-nique"),
    ("He checks his email", "Il vérifie ses emails"),
    ("They go to the gym", "Ils vont à la salle de sport"),
    ("The moon shines brightly", "La lune brille intensément"),
    ("She catches the bus", "Elle attrape le bus"),
    ("We greet our neighbors", "Nous saluons nos voisins"),
    ("He combs his hair", "Il se peigne les cheveux"),
    ("They wave goodbye", "Ils font un signe d'adieu")

]

####split into English and French
english_sentences, french_sentences = zip(*english_to_french)

###Add SOS and EOS tokens to French sentences
french_sentences = ['<SOS> ' + sent + ' <EOS>' for sent in french_sentences]

# Tokenization
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_sentences)
eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)
eng_vocab_size = len(eng_tokenizer.word_index) + 1

fr_tokenizer = Tokenizer(filters='')
fr_tokenizer.fit_on_texts(french_sentences)

####Ensure <SOS> and <EOS> are in the vocabulary
if '<SOS>' not in fr_tokenizer.word_index:
    fr_tokenizer.word_index['<SOS>'] = len(fr_tokenizer.word_index) + 1
if '<EOS>' not in fr_tokenizer.word_index:
    fr_tokenizer.word_index['<EOS>'] = len(fr_tokenizer.word_index) + 1
fr_sequences = fr_tokenizer.texts_to_sequences(french_sentences)
fr_vocab_size = len(fr_tokenizer.word_index) + 1

#Debug: Inspect the vocabulary
print("French tokenizer word_index:", fr_tokenizer.word_index)

######Padding sequences
max_eng_len = max(len(seq) for seq in eng_sequences)
max_fr_len = max(len(seq) for seq in fr_sequences)
eng_padded = pad_sequences(eng_sequences, maxlen=max_eng_len, padding='post')
fr_padded = pad_sequences(fr_sequences, maxlen=max_fr_len, padding='post')

####Prepare decoder input and target data
decoder_input_data = fr_padded[:, :-1]  # Exclude <EOS>
decoder_target_data = fr_padded[:, 1:]  # Exclude <SOS>

####Model Parameters
embedding_dim = 256
units = 512

#Encoder
encoder_inputs = Input(shape=(max_eng_len,))
enc_embedding = Embedding(eng_vocab_size, embedding_dim)(encoder_inputs)
encoder_gru = GRU(units, return_state=True)
encoder_outputs, encoder_state = encoder_gru(enc_embedding)

#Decoder
decoder_inputs = Input(shape=(max_fr_len - 1,))
dec_embedding = Embedding(fr_vocab_size, embedding_dim)(decoder_inputs)
decoder_gru = GRU(units, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(dec_embedding, initial_state=encoder_state)
decoder_dense = Dense(fr_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

##################Train and evaluate on the entire dataset
history = model.fit(
    [eng_padded, decoder_input_data],
    np.expand_dims(decoder_target_data, -1),
    batch_size=32,
    epochs=50,
    validation_data=([eng_padded, decoder_input_data], np.expand_dims(decoder_target_data, -1)),
    verbose=1
)

#######Report results
print("Training Loss:", history.history['loss'][-1])
print("Validation Loss (Entire Dataset):", history.history['val_loss'][-1])
print("Validation Accuracy (Entire Dataset):", history.history['val_accuracy'][-1])

#####Inference Setup
encoder_model = Model(encoder_inputs, encoder_state)

decoder_state_input = Input(shape=(units,))
decoder_inputs_single = Input(shape=(1,))
dec_embedding_single = Embedding(fr_vocab_size, embedding_dim)(decoder_inputs_single)
decoder_outputs, decoder_state = decoder_gru(dec_embedding_single, initial_state=decoder_state_input)
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs_single, decoder_state_input], [decoder_outputs, decoder_state])

#3####Translation Function (fixed to stop at first <EOS> and debug)
def translate(sentence):
    seq = eng_tokenizer.texts_to_sequences([sentence])
    seq = pad_sequences(seq, maxlen=max_eng_len, padding='post')
    state = encoder_model.predict(seq, verbose=0)

    target_seq = np.array([[fr_tokenizer.word_index['<SOS>']]])
    output_sentence = []

    for _ in range(max_fr_len):
        output_tokens, state = decoder_model.predict([target_seq, state], verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, 0, :])
        sampled_word = fr_tokenizer.index_word.get(sampled_token_index, '')

        if sampled_word == '<EOS>' or sampled_word == '':
            break
        output_sentence.append(sampled_word)
        target_seq = np.array([[sampled_token_index]])

    return ' '.join(output_sentence)

#####Qualitative Validation
test_sentences = [
    "I am cold",
    "She is happy",
    "We love music",
    "He drives a blue car",
    "The sun is shining"
]

print("\nQualitative Validation:")
for sent in test_sentences:
    translation = translate(sent)
    print(f"English: {sent} -> French: {translation}")

tf.config.run_functions_eagerly(False)

French tokenizer word_index: {'<sos>': 1, '<eos>': 2, 'le': 3, 'la': 4, 'il': 5, 'elle': 6, 'nous': 7, 'ils': 8, 'les': 9, 'de': 10, 'à': 11, 'des': 12, 'au': 13, 'dans': 14, 'du': 15, 'une': 16, 'un': 17, 'se': 18, 'font': 19, 'est': 20, 'chat': 21, 'dort': 22, 'soleil': 23, 'musique': 24, 'jouent': 25, 'porte': 26, 'ensemble': 27, 'étudie': 28, 'bruyamment': 29, 'chante': 30, 'matin': 31, 'pour': 32, 'brille': 33, 'livres': 34, 'chaque': 35, 'film': 36, 'dîner': 37, 'voiture': 38, 'visitent': 39, 'cuisine': 40, 'regardons': 41, 'en': 42, 'voyagent': 43, 'table': 44, 'danse': 45, 'avec': 46, 'dur': 47, 'fleurs': 48, 'écrit': 49, 'son': 50, 'peint': 51, 'parc': 52, 'train': 53, 'bébé': 54, 'répare': 55, 'café': 56, 'doucement': 57, 'bus': 58, 'vélo': 59, 'nos': 60, 'ses': 61, "j'ai": 62, 'froid': 63, 'tu': 64, 'es': 65, 'fatigué': 66, 'a': 67, 'faim': 68, 'heureuse': 69, 'sommes': 70, 'amis': 71, 'sont': 72, 'étudiants': 73, 'aimons': 74, 'parle': 75, 'français': 76, 'couramment': 77, 